# Bridge L3.M5.1 → L3.M5.2 — Incremental Indexing to Data Pipelines

**Track:** CCC Level 2 - Module 5: Production Data Management  
**Purpose:** Validation notebook for bridge readiness checks  
**Duration:** 8-10 minutes  
**Bridge Type:** Within-Module  

---

## 1) RECAP — What M5.1 Actually Shipped

In **M5.1: Incremental Indexing & Updates**, you accomplished:

✓ **Change detection system** — Uses SHA-256 checksums to detect document modifications in under 2 seconds (tested on 10,000 documents)

✓ **Targeted Pinecone updates** — Deletes only modified chunks and inserts new versions, reducing update time from 20 minutes to 3-5 seconds (95% faster)

✓ **Version tracking with rollback** — Maintains history of last 5 versions per document, enabling instant rollback when updates break production

✓ **Atomic two-phase commit** — Ensures index never enters inconsistent state, even if process crashes mid-update

**Key Achievement:** Transformed a system requiring $50 and 20 minutes for every update into one costing $0.10 and taking 3-5 seconds.

In [ ]:
import os
import json
from pathlib import Path

# Check for incremental update script
script_exists = os.path.exists("incremental_update.py")
print(f"✓ Incremental update script exists: {script_exists}")

# Expected: 
# ✓ Incremental update script exists: True
# ⚠️ If False: Script not found (implement in M5.1)

if not script_exists:
    print("⚠️ Skipping (no incremental_update.py found)")

In [ ]:
import hashlib

# Simulate checksum calculation
def calculate_checksum(content):
    return hashlib.sha256(content.encode()).hexdigest()

# Test change detection
doc_v1 = "Original policy content"
doc_v2 = "Updated policy content"

checksum_v1 = calculate_checksum(doc_v1)
checksum_v2 = calculate_checksum(doc_v2)

print(f"✓ SHA-256 checksum calculation working")
print(f"  V1 checksum: {checksum_v1[:16]}...")
print(f"  V2 checksum: {checksum_v2[:16]}...")
print(f"  Changed: {checksum_v1 != checksum_v2}")

# Expected:
# ✓ SHA-256 checksum calculation working
# Changed: True

In [ ]:
# Check for persistent version tracking metadata
checksums_file = "checksums.json"
checksums_exists = os.path.exists(checksums_file)

print(f"✓ Checksums metadata file exists: {checksums_exists}")

if checksums_exists:
    with open(checksums_file, 'r') as f:
        metadata = json.load(f)
    print(f"  Documents tracked: {len(metadata)}")
    print(f"  Sample: {list(metadata.keys())[:2]}")
else:
    print("⚠️ Skipping (no checksums.json found)")
    # Create sample structure for reference
    sample_metadata = {
        "policy_2024.pdf": {
            "current_checksum": "abc123...",
            "version_history": ["v1_hash", "v2_hash", "v3_hash"]
        }
    }
    print(f"  Expected structure: {json.dumps(sample_metadata, indent=2)[:80]}...")

# Expected:
# ✓ Checksums metadata file exists: True
# Documents tracked: 50

In [ ]:
# Check for update logs showing incremental behavior
log_file = "update_log.txt"
log_exists = os.path.exists(log_file)

print(f"✓ Update log exists: {log_exists}")

if log_exists:
    with open(log_file, 'r') as f:
        lines = f.readlines()[:5]  # Read first 5 lines
    print(f"  Log entries: {len(lines)}")
    for line in lines:
        print(f"    {line.strip()}")
else:
    print("⚠️ Skipping (no update_log.txt found)")
    print("\n  Expected format:")
    print("  [2025-11-02 02:00:01] Starting incremental update")
    print("  [2025-11-02 02:00:03] Detected 3 changed documents")
    print("  [2025-11-02 02:00:18] Processed policy_2024.pdf (5 chunks)")
    print("  [2025-11-02 02:00:24] Update complete: 3 docs, 21 seconds")

# Expected:
# ✓ Update log exists: True
# Shows targeted updates (not full re-index)

## 6) CALL-FORWARD — What M5.2 Will Introduce and Why\n\n### The Problem: Who Runs It at 2 AM?\n\nYour incremental indexing works perfectly—detects changes instantly, updates surgically, tracks versions. But it's still a **manual process**.\n\n**Current state:** You run `python incremental_update.py` manually\n\n**The burn rate:**\n- **Time cost:** 30 minutes per day checking for updates = 180 hours per year\n- **Opportunity cost:** While running updates, you're not building features = $9,000 lost productivity (@$50/hour)\n- **Risk cost:** Miss one update = compliance violation = legal exposure\n\n**Total hidden cost:** $9,000+ per year in manual labor\n\n---\n\n### What M5.2: Data Pipelines & Orchestration Will Add\n\n**1. Automated scheduling with Apache Airflow**\n   → No more manual triggers—pipelines run daily, hourly, or on-demand with zero human intervention\n\n**2. Parallel processing that cuts time from 40 minutes to 8 minutes**\n   → For 5,000 documents, process 4-8 at a time instead of sequentially\n\n**3. Graceful error handling with automatic retries and alerting**\n   → One failed document doesn't crash entire pipeline, Slack alerts on failures\n\n---\n\n### The Bridge Question\n\n**\"Your incremental indexing works perfectly—but who runs it at 2 AM when documents update overnight?\"**\n\nM5.2 will answer this by automating your data refresh pipeline.\n\n---\n\n**Next:** [M5.2 Concept - Data Pipelines & Orchestration](https://github.com/yesvisare/ccc_l2_bridge)"

## 5) Readiness Check #4 — Successfully Updated at Least One Document Without Full Re-Index

**Check:** Update log shows targeted update (not full corpus reprocessing)  
**Impact:** Confirms incremental logic works before adding orchestration layer

**What to verify:**
- Update log exists showing partial updates
- Only changed documents were processed (not all documents)
- Update time < 10 seconds (not minutes)

## 4) Readiness Check #3 — Version Tracking Metadata Stored Persistently

**Check:** `checksums.json` file exists with version history for all documents  
**Impact:** Prevents data loss during Airflow migration (metadata must persist across runs)

**What to verify:**
- Checksum metadata file exists
- Contains version history (last 5 versions per document)
- Persists across script runs (not in-memory only)

## 3) Readiness Check #2 — Change Detection Using Checksums

**Check:** Modify one document, verify checksum changes and triggers update  
**Impact:** Saves 4+ hours debugging why "automation isn't detecting changes"

**What to verify:**
- Checksum calculation uses SHA-256 (not timestamps)
- Modified document produces different checksum
- Change detection triggers correctly

## 2) Readiness Check #1 — Incremental Indexing Implemented and Tested

**Check:** Run incremental update on test corpus, verify only changed docs process  
**Impact:** Saves 2 hours debugging Airflow if base pipeline is broken

**What to verify:**
- Incremental update script exists and runs successfully
- Only modified documents are processed (not full re-index)
- Update completes in reasonable time (seconds, not minutes)